In [97]:
import configparser
import spotipy
from utils.auth_init import auth_init

In [98]:
# Check key folder and if does not exist create empty templates for keys
auth_init()

auth/spotify_keys.ini already exists.
auth/roblox_keys.ini already exists.


### Spotify Credentials

In [99]:
from spotipy.oauth2 import SpotifyClientCredentials

# Read Spotify credentials from a configuration file
config = configparser.ConfigParser()
config.read('auth/spotify_keys.ini')

# Variables for Spotify Data
client_id = config['SpotifyAPI']['client_id']
client_secret = config['SpotifyAPI']['client_secret']
username = config['SpotifyAPI']['username']
redirectURI = config['SpotifyAPI']['redirect']

# Creeating Data Authentication Manager for Spotify Credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Roblox Authentication

In [100]:
import robloxpy
import requests
import webbrowser
import spotipy
import spotipy.util as util
import json
import base64
from roblox import Client
import roblox.thumbnails as thumbnail
import asyncio
from utils.roblox_functions import *

# Read Roblox Player Cookie
RobloxAuth = RobloxAuthConfig('auth/roblox_keys.ini')
cookie = RobloxAuth.cookie

# Setting Roblox Cookie to Client using Ro.Py
client = Client(cookie)

async def main():
    user = await client.get_authenticated_user()
    # *** ID CAN BE PRINTED *** Test to see if you are Ro.Py Authenticated
    print("ID:", user.id)
    
await main()

ID: 7520736841


### Roblox Username Search

In [101]:
# Input | Search Roblox Usernames
desired_name = "Funky"
outputted_names = []

async def main():
    # Search the username and amount of results
    users = client.user_search(desired_name, max_items =10)
    async for user in users:
        # Add to list
        outputted_names.append(user.name)
        outputted_names.append(user.id)
await main()

# Format list to Dictonary to access Name & ID
formatted_names = []
for i in range(0, len(outputted_names), 2):
    formatted_names.append({'name': outputted_names[i], 'id': outputted_names[i + 1]})

formatted_names

[{'name': 'funky', 'id': 115158}]

### Roblox Thumbnail Finder

In [102]:
# Pick a user: this could be random but in this case its top result
desired_user = formatted_names[0]
user = await client.get_user(desired_user["id"])

# Get the avatar thumbnail found on their profile
user_thumbnails = await client.thumbnails.get_user_avatar_thumbnails(
    users=[user],
    type=thumbnail.AvatarThumbnailType.full_body,
    size=(420, 420)
)

# Print the URL of thumbnail
if len(user_thumbnails) > 0:
    user_thumbnail = user_thumbnails[0]
    user_image = user_thumbnail.image_url
    print(user_image)

https://tr.rbxcdn.com/30DAY-Avatar-5F0AB89B2E5D51F8701D55CE31295026-Png/420/420/Avatar/Png/noFilter


### Spotify Scoping

In [103]:
# Spotify Sscopes (INCLUDE ugc-image-upload)
scope = "user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public playlist-modify-private ugc-image-upload"
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

sp = spotipy.Spotify(auth=token)

### Spotify Search by Username

In [104]:
# Prints Username of Robloxian
print(formatted_names[0]["name"])

# Search Spotify based on the Robloxians Username
track_results = sp.search(q=desired_user["name"], type="track", limit = 10)

funky


### Song URI Grabber

In [105]:
song_data = track_results["tracks"]["items"]

# Get song URIs into a list
song_uris = []
for song in song_data:
    song_uris.append(song["uri"])

### Playlist Generator

In [106]:
my_playlist = sp.user_playlist_create(user=username, name=desired_user["name"], public=True,
                                      description=f"Playlist for the Robloxian {desired_user["name"]}!")
# Add the songs
results = sp.user_playlist_add_tracks(username, my_playlist['id'], song_uris)
print(results)

# Convert the user thumbnail through base64 enconde
def get_as_base64(url):
    response = requests.get(url)
    return base64.b64encode(response.content).decode('utf-8')

get_as_base64(user_image)

# Get image and upload the image as playlist cover
user_image_base64 = get_as_base64(user_image)
sp.playlist_upload_cover_image(my_playlist['id'], user_image_base64)

# Open Spotify
webbrowser.open(my_playlist['external_urls']['spotify'])

{'snapshot_id': 'AAAAAqgo31PfglT7ZxLZW4oF16WkhaW6'}


True

In [107]:
import pandas as pd
df = pd.DataFrame(track_results['tracks']['items'])

In [108]:
df['artists'][0][0]['name']

'Dave'

In [109]:
df['artists'][1]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0lwRI7lvmlRY5DiA5Xa6wQ'},
  'href': 'https://api.spotify.com/v1/artists/0lwRI7lvmlRY5DiA5Xa6wQ',
  'id': '0lwRI7lvmlRY5DiA5Xa6wQ',
  'name': 'Lipps Inc.',
  'type': 'artist',
  'uri': 'spotify:artist:0lwRI7lvmlRY5DiA5Xa6wQ'}]

In [110]:
def spotify_tracks_to_df(track_results):

    def clean_artist(list_of_artists):
        just_names = [d['name'] for d in list_of_artists]
        results_string = " & ".join(just_names)

        return results_string
    

    df = pd.DataFrame(track_results['tracks']['items'])
    df['clean_artist_names'] = df['artists'].apply(lambda x: clean_artist(x))
    clean_df = df[['name','clean_artist_names']]
    clean_df = clean_df.rename(columns = {'clean_artist_names':'artists','name':'song'})

    return clean_df

In [111]:
spotify_df = spotify_tracks_to_df(track_results)
spotify_df

,song,artists
0,Funky Friday,Dave & Fredo
1,Funkytown,Lipps Inc.
2,Funkytown,Lipps Inc.
3,Thiago Silva,Dave & AJ Tracey
4,Funky Friday,Dave & Fredo
5,Play That Funky Music,Wild Cherry
6,Funkytown - Single Version,Lipps Inc.
7,Funky Cold Medina,Tone-Loc
8,Funky Kingston,Toots & The Maytals
9,Funky Shit,The Prodigy


In [112]:
# get all lua paths
import os
# Specify the directory
directory = 'lua_scripts/'

# Get all file paths
lua_file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
lua_file_paths

['lua_scripts/display_script.lua',
 'lua_scripts/parameter_classes.lua',
 'lua_scripts/ui_script.lua']

In [113]:
import re

# Read the Lua script from the file
file_path = "lua_scripts/parameter_classes.lua"
with open(file_path, 'r') as file:
    lua_script = file.read()

text = lua_script

# Regex pattern to match all song entries
pattern = r'{artist = "([^"]+)", song = "([^"]+)"}'

# Find all matches
matches = list(re.finditer(pattern, text))

# Substitute each match one by one
for i,match in enumerate(matches):
    artist, song = match.groups()
    print(f"Matched artist: {artist}, song: {song}")
    
    # Modify text: replace current match with a custom substitution
    text = re.sub(re.escape(match.group(0)), f'{{artist = "{spotify_df['artists'][i]}", song = "{spotify_df['song'][i]}"}}', text, count=1)

print("\nModified Text:\n")
print(text)


Matched artist: Artist 1, song: Song 1
Matched artist: Artist 2, song: Song 2
Matched artist: Artist 3, song: Song 3
Matched artist: Artist 4, song: Song 4
Matched artist: Artist 5, song: Song 5
Matched artist: Artist 6, song: Song 6
Matched artist: Artist 7, song: Song 7
Matched artist: Artist 8, song: Song 8
Matched artist: Artist 9, song: Song 9
Matched artist: Artist 10, song: Song 10

Modified Text:

-- ParameterStart
local data = {
	{artist = "Dave & Fredo", song = "Funky Friday"},
	{artist = "Lipps Inc.", song = "Funkytown"},
	{artist = "Lipps Inc.", song = "Funkytown"},
	{artist = "Dave & AJ Tracey", song = "Thiago Silva"},
	{artist = "Dave & Fredo", song = "Funky Friday"},
	{artist = "Wild Cherry", song = "Play That Funky Music"},
	{artist = "Lipps Inc.", song = "Funkytown - Single Version"},
	{artist = "Tone-Loc", song = "Funky Cold Medina"},
	{artist = "Toots & The Maytals", song = "Funky Kingston"},
	{artist = "The Prodigy", song = "Funky Shit"},
}
-- ParameterEnd


In [114]:
with open("lua_scripts/display_script.lua", "r") as display_file:
    display_script = display_file.read()

with open("lua_scripts/ui_script.lua", "r") as ui_file:
    ui_script = ui_file.read()

ui_script = text + ui_script
display_script = text + display_script

In [115]:
# Get script instance
main_object_path = ("ServerScriptService/DisplayIterator")
instance_id = find_instance_id(RobloxAuth,object_path=main_object_path)

ui_object_path = ("StarterPlayer/StarterCharacterScripts/UpdateUI")
instance_id_2 = find_instance_id(RobloxAuth,object_path=ui_object_path)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

At level: ServerScriptService


 50%|██████████████████████████████████████████                                          | 1/2 [00:05<00:05,  5.37s/it]

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/108471ac-b6e3-9bdc-0733-9ba500000350', 'hasChildren': True, 'engineInstance': {'Id': '108471ac-b6e3-9bdc-0733-9ba500000350', 'Parent': '36569f41-47e1-34fd-073b-97df00000001', 'Name': 'ServerScriptService', 'Details': {}}}
At level: DisplayIterator


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.37s/it]


...

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/108471ac-b6e3-9bdc-0733-9ba500000325', 'hasChildren': True, 'engineInstance': {'Id': '108471ac-b6e3-9bdc-0733-9ba500000325', 'Parent': '36569f41-47e1-34fd-073b-97df00000001', 'Name': 'StarterPlayer', 'Details': {}}}
At level: StarterCharacterScripts


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:10<00:05,  5.35s/it]

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/108471ac-b6e3-9bdc-0733-9ba50000037b', 'hasChildren': True, 'engineInstance': {'Id': '108471ac-b6e3-9bdc-0733-9ba50000037b', 'Parent': '108471ac-b6e3-9bdc-0733-9ba500000325', 'Name': 'StarterCharacterScripts', 'Details': {}}}
At level: UpdateUI


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.35s/it]

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/6fee2929-cab0-0158-073a-4bad00016972', 'hasChildren': False, 'engineInstance': {'Id': '6fee2929-cab0-0158-073a-4bad00016972', 'Parent': '108471ac-b6e3-9bdc-0733-9ba50000037b', 'Name': 'UpdateUI', 'Details': {'LocalScript': {'Enabled': True, 'Source': ''}}}}
Final instance id returned: 6fee2929-cab0-0158-073a-4bad00016972


In [116]:
#print(display_script)

In [117]:
# Updating script

propertyName = "Source" # API defined - not sure if this ever changes
propertyValue = display_script  # The new script content
instanceType = "Script"  # Name of script instance (I think)
update_instance(RobloxAuth,instance_id,propertyName=propertyName,propertyValue=propertyValue,instanceType=instanceType)

propertyName = "Source"
propertyValue = ui_script 
instanceType = "LocalScript" 
update_instance(RobloxAuth,instance_id_2,propertyName=propertyName,propertyValue=propertyValue,instanceType=instanceType)

# This won't work if you have the live script update opened, check the response text. Might add easy response text later

{'Script': {'Source': '-- ParameterStart\nlocal data = {\n\t{artist = "Dave & Fredo", song = "Funky Friday"},\n\t{artist = "Lipps Inc.", song = "Funkytown"},\n\t{artist = "Lipps Inc.", song = "Funkytown"},\n\t{artist = "Dave & AJ Tracey", song = "Thiago Silva"},\n\t{artist = "Dave & Fredo", song = "Funky Friday"},\n\t{artist = "Wild Cherry", song = "Play That Funky Music"},\n\t{artist = "Lipps Inc.", song = "Funkytown - Single Version"},\n\t{artist = "Tone-Loc", song = "Funky Cold Medina"},\n\t{artist = "Toots & The Maytals", song = "Funky Kingston"},\n\t{artist = "The Prodigy", song = "Funky Shit"},\n}\n-- ParameterEnd-- INSERT PARAMETERS\n\nlocal displayGroup = workspace.Displays\nlocal display_no = 0\t\n\nlocal displays = {}\nfor _, display in ipairs(displayGroup:GetChildren()) do\n\tif string.match(display.Name, "SpotifyDisplay%d+") then\n\t\ttable.insert(displays, {\n\t\t\tartist_display = display["Artist Display"].SurfaceGui.TextLabel,\n\t\t\tsong_display = display["Song Display"

In [118]:
print(RobloxAuth.place_id)

106470241725015


In [119]:
# Request header
api_keyHeaderKey = "x-api-key"
contentTypeHeaderKey = "Content-type"
contentTypeHeaderValue = "application/json"

# Endpoint URL for Update Instance method
publishURL = 'https://apis.roblox.com/universes/v1/%s/places/%s/versions?versionType=Published'
def publish_experience():
    url = f"https://apis.roblox.com/universes/v1/{RobloxAuth.universe_id}/places/{RobloxAuth.place_id}/versions"  # f-string
    print(url)
    headerData = {api_keyHeaderKey: RobloxAuth.api_key,
    contentTypeHeaderKey: contentTypeHeaderValue}
    data = {"versionType": "Published"}
    return requests.patch(url, headers = headerData, json=data)

publish_experience()
#update_instance(RobloxAuth,instance_id_2,propertyName=propertyName,propertyValue=propertyValue,instanceType=instanceType)

https://apis.roblox.com/universes/v1/6737450158/places/106470241725015/versions


<Response [404]>

In [120]:
api_keyHeaderKey = "x-api-key"
contentTypeHeaderKey = "Content-type"
contentTypeHeaderValue = "application/json"
data_file_path = "/path/to/your/file.rbxl"

def publish_experience(RobloxAuth):
    url = f"https://apis.roblox.com/universes/v1/{RobloxAuth.universe_id}/places/{RobloxAuth.place_id}/versions"
    headerData = {api_keyHeaderKey: RobloxAuth.api_key,
        contentTypeHeaderKey: contentTypeHeaderValue}
    data = {"versionType": "Published"}

    try:
        response = requests.post(url, headers=headerData, json=data)  # Use POST
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        print(f"Publish successful: {response.json()}")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error publishing: {e}")
        if response: # Print the error response from Roblox
            print(f"Roblox error response: {response.text}")
        return None

publish_response = publish_experience(RobloxAuth)

Error publishing: 400 Client Error: Bad Request for url: https://apis.roblox.com/universes/v1/6737450158/places/106470241725015/versions
